In [ ]:
import sys
import os
sys.path.append('/home/kai/DAST/network')
from enums import NormType, TrainType
from train import DASTModel, DASTTF
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

## DL

In [ ]:
selected_indices = [1, 3, 5, 7, 9, 10, 14]
# selected_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
FEATURE_LEN = len(selected_indices)
FEATURE_SIZE = 20
EMBEDD = 20
HYPER_PARAMETERS = {
    # model parameter
    'batch_size': 256,
    'dim_val': FEATURE_SIZE,
    'dim_attn': EMBEDD,
    'dim_val_t': FEATURE_SIZE,
    'dim_attn_t': EMBEDD,
    'dim_val_s': FEATURE_SIZE,
    'dim_attn_s': EMBEDD,
    'n_heads': 4,
    'n_decoder_layers': 1,
    'n_encoder_layers': 2,
    'lr': 1e-3,
    'epochs': 100,
    'time_step': 40,
    # limit how many last input used, important!
    'dec_seq_len': 6,
    'output_sequence_length': 1,
    'feature_len': FEATURE_LEN,
    'debug': True
    # else parameter
}

In [ ]:
DATA_PATH = '../../../data/10FEMTO/processed_data/'
TRAIN_DATASETS = ['Bearing1_1', 'Bearing1_2']
TEST_DATASET = ['Bearing1_3', 'Bearing1_4', 'Bearing1_5', 'Bearing1_6', 'Bearing1_7']
# TRAIN_DATASETS = ['Bearing2_1', 'Bearing2_2']
# TEST_DATASET = ['Bearing2_3', 'Bearing2_4', 'Bearing2_5', 'Bearing2_6', 'Bearing2_7']
# TRAIN_DATASETS = ['Bearing3_1', 'Bearing3_2']
# TEST_DATASET = ['Bearing3_3']

MODEL_SAVE_NAME = f'Bearing{TRAIN_DATASETS[0][7]}_pretrain_{len(selected_indices)}_norm_test'
NORM_TYPE = NormType.NO_NORM
TRAIN_TYPE = TrainType.DL
MODEL_SAVE_PATH = '../../../model/norm'
IS_SAVE_MODEL = False

In [ ]:
dast_model = DASTModel(train_datasets=TRAIN_DATASETS, test_dataset=TEST_DATASET, data_path=DATA_PATH, norm_type=NORM_TYPE, hyper_parameters=HYPER_PARAMETERS, model_save_path=MODEL_SAVE_PATH, model_save_name=MODEL_SAVE_NAME, is_save_model=IS_SAVE_MODEL, train_type=TRAIN_TYPE)

In [ ]:
dast_model.main(selected_indices)

In [ ]:
dast_model.plt_loss_list()
dast_model.plt_best_predict()
dast_model.plt_last_predict_vs_y()

## TL

In [ ]:
selected_indices = [1, 3, 5, 7, 9, 10, 14]
# selected_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
FEATURE_LEN = len(selected_indices)
FEATURE_SIZE = 20
EMBEDD = 10
HYPER_PARAMETERS = {
    # model parameter
    'batch_size': 256,
    'dim_val': FEATURE_SIZE,
    'dim_attn': EMBEDD,
    'dim_val_t': FEATURE_SIZE,
    'dim_attn_t': EMBEDD,
    'dim_val_s': FEATURE_SIZE,
    'dim_attn_s': EMBEDD,
    'n_heads': 4,
    'n_decoder_layers': 1,
    'n_encoder_layers': 2,
    'lr': 1e-3,
    'epochs': 10,
    'time_step': 40,
    # limit how many last input used, important!
    'dec_seq_len': 6,
    'output_sequence_length': 1,
    'feature_len': FEATURE_LEN,
    'debug': True
}

In [ ]:
DATA_PATH = '../../../data/10FEMTO/processed_data/'
PRETRAIN_DATASET = 'Bearing2'
FINETUNE_DATASET = 'Bearing3'

TRAIN_DATASETS = ['Bearing1_1', 'Bearing1_2']
TEST_DATASET = ['Bearing1_3', 'Bearing1_4', 'Bearing1_5', 'Bearing1_6', 'Bearing1_7']
# TRAIN_DATASETS = ['Bearing2_1', 'Bearing2_2']
# TEST_DATASET = ['Bearing2_3', 'Bearing2_4', 'Bearing2_5', 'Bearing2_6', 'Bearing2_7']
# TRAIN_DATASETS = ['Bearing3_1', 'Bearing3_2']
# TEST_DATASET = ['Bearing3_3']

MODEL_SAVE_PATH = '../../../model'
MODEL_SAVE_NAME = f'{PRETRAIN_DATASET}_pretrain_{FINETUNE_DATASET}_finetune_7'
PRETRAIN_MODEL_PATH = '../../../model'
PRETRAIN_MODEL_NAME = f'{PRETRAIN_DATASET}_pretrain_7'
IS_SAVE_MODEL = False
NORM_TYPE = NormType.NO_NORM
TRAIN_TYPE = TrainType.TL

In [ ]:
pretrain_model = DASTTF(HYPER_PARAMETERS, PRETRAIN_MODEL_PATH, PRETRAIN_MODEL_NAME).pretrained_model

In [ ]:
dast_model = DASTModel(train_datasets=TRAIN_DATASETS, test_dataset=TEST_DATASET, data_path=DATA_PATH, norm_type=NORM_TYPE, hyper_parameters=HYPER_PARAMETERS, model_save_path=MODEL_SAVE_PATH, model_save_name=MODEL_SAVE_NAME, is_save_model=IS_SAVE_MODEL, train_type=TRAIN_TYPE)
dast_model.set_model(pretrain_model)

In [ ]:
dast_model.main(selected_indices)

In [ ]:
dast_model.plt_loss_list()
dast_model.plt_best_predict()
dast_model.plt_last_predict_vs_y()